In [1]:
%load_ext autoreload
%autoreload 2
from context import *

# Stablizer Formalism (`stabilizer`)

### General Idea: State-Map Duality

Every stabilizer state $\rho$ is dual to a Clifford unitary $U$, such that the state can be generated from the zero state $|00\cdots0\rangle$ as
$$\rho = U|00\cdots0\rangle\langle 00\cdots0|U^\dagger$$.
Both $\rho$ and $U$ describes a stabilizer code:
* $\rho$ is a projection operator that specifies the code subspace of the stabilizer code.
* $U$ is the encoding Clifford unitary that encodes the logical + syndrome qubits to the physical qubits in the stabilizer code.

The package `stabilizer` (based on `paulialg`) provides related functions to represent stabilizer states and Clifford maps. There are two classes defined in this package.

* `stabilizer.CliffordMap`. Since the Clifford unitary $U$ maps Pauli operators to Pauli operators, it is sufficient to specify a Clifford unitary by how each single-qubit Pauli operator transforms under the unitary. Such transformation rules are stored in a table called the Clifford map.

* `stabilizer.StabilizerState`. The stabilizer state is specified by a set of stabilizers and the corresponding destabilizers. Using the binary representation of Pauli operators, they can be stored in a table, called the stabilizer tableau. 

Since both classes need to store a table of Pauli operators, they are both realized as subclasses of `paulialg.PauliList`.

## Basic Usage

### Constructors

#### Construct Clifford Maps

`identity_map(N)` constructs an identity Clifford map on $N$ qubits.

In [6]:
identity_map(4)

CliffordMap(
  X0-> +XIII
  Z0-> +ZIII
  X1-> +IXII
  Z1-> +IZII
  X2-> +IIXI
  Z2-> +IIZI
  X3-> +IIIX
  Z3-> +IIIZ)

`random_pauli_map(N)` samples a random Clifford map made of random single-qubit Clifford gates on $N$ qubits, i.e. $U=\prod_i U_i\in\mathrm{Cl}(2)^N$. Each realization specifies a random local Pauli basis.

In [7]:
random_pauli_map(4)

CliffordMap(
  X0-> -ZIII
  Z0-> +XIII
  X1-> -IZII
  Z1-> -IYII
  X2-> -IIZI
  Z2-> -IIYI
  X3-> +IIIZ
  Z3-> -IIIY)

`random_clifford_map(N)` samples a globally random Clifford map on $N$ qubits, i.e. $U\in\mathrm{Cl}(2^N)$. Each realization specifies a random global stabilizer basis.

In [8]:
random_clifford_map(4)

CliffordMap(
  X0-> +IIZI
  Z0-> +YZXI
  X1-> +YZIX
  Z1-> +YIIZ
  X2-> +YZII
  Z2-> -IYZZ
  X3-> -ZXIX
  Z3-> -XXIX)

`clifford_rotation_map(N)` constructs a Clifford map based for a Clifford rotation given its generator.

In [9]:
clifford_rotation_map('-XXYZ')

CliffordMap(
  X0-> +XIII
  Z0-> +YXYZ
  X1-> +IXII
  Z1-> +XYYZ
  X2-> +XXZZ
  Z2-> -XXXZ
  X3-> -XXYY
  Z3-> +IIIZ)

#### Construct Stabilizer States

`maximally_mixed_state(N)` constructs a $N$-qubit maximally mixed state (by setting the density matrix to full rank).
$$\rho=2^{-N}\mathbb{1}.$$

In [10]:
maximally_mixed_state(4)

StabilizerState()

`zero_state(N)` constructs a $N$-qubit all-zero state 
$$\rho=|0\cdots0\rangle\langle 0\cdots0|=\prod_{i}\frac{1+Z_i}{2}.$$

In [11]:
zero_state(4)

StabilizerState(
   +ZIII
   +IZII
   +IIZI
   +IIIZ)

`one_state(N)` constructs a $N$-qubit all-one state 
$$\rho=|1\cdots1\rangle\langle 1\cdots1|=\prod_{i}\frac{1-Z_i}{2}.$$

In [12]:
one_state(4)

StabilizerState(
   -ZIII
   -IZII
   -IIZI
   -IIIZ)

`ghz_state(N)` constructs a $N$-qubit GHZ state
$$\rho = |\Psi\rangle\langle\Psi|, \qquad \text{with }|\Psi\rangle=\frac{1}{\sqrt{2}}(|0\cdots0\rangle+|1\cdots1\rangle).$$

In [13]:
ghz_state(4)

StabilizerState(
   +XXXX
   +IIZZ
   +IZZI
   +ZZII)

`random_pauli_map(N)` samples a $N$ qubit random Pauli state.
$$\rho=U|0\cdots0\rangle\langle 0\cdots0|U^\dagger,\qquad\text{with }U\in \mathrm{Cl}(2)^N.$$

In [14]:
random_pauli_state(4)

StabilizerState(
   -YIII
   +IZII
   +IIYI
   -IIIX)

`random_clifford_map(N)` samples a $N$ qubit random Clifford (random stabilizer) state.
$$\rho=U|0\cdots0\rangle\langle 0\cdots0|U^\dagger,\qquad\text{with }U\in \mathrm{Cl}(2^N).$$

In [15]:
random_clifford_state(4)

StabilizerState(
   +XYIZ
   +YIYX
   +YIIX
   +YXYI)

`stabilizer_state(...)` is a universal constructor of stabilizer state by specifying all stabilizers.

In [4]:
stabilizer_state('XXY','-YYI')

StabilizerState(
   -YYI
   +XXY)

A hack to inspect the full stabilizer tableau is by converting `StabilizerState` to `PauliList` by

In [5]:
stabilizer_state('XXY','-YYI')[:]

 +ZZI
 -YYI
 +XXY
 +ZXZ
 +ZIZ
 +IIZ

User need to ensure that stabilizers commute with each other, otherwise an error will be raised.

In [6]:
stabilizer_state('XXY','-YYI','IZZ')

ValueError: stabilizers must all commute with each other.

#### State-Map Conversion

Stabilizer states and Clifford maps can be mapped to each other.

In [10]:
rho = stabilizer_state('XXY','-YYI')
rho

StabilizerState(
   -YYI
   +XXY)

In [8]:
rho.to_map()

CliffordMap(
  X0-> +ZXZ
  Z0-> +ZZI
  X1-> +ZIZ
  Z1-> -YYI
  X2-> +IIZ
  Z2-> +XXY)

In [9]:
rho.to_map().to_state()

StabilizerState(
   +ZZI
   -YYI
   +XXY)

* `.to_map()` and `.to_state()` will make new copies of Pauli string data in the memory.
* the information about the rank of the density matrix is lost in the Clifford map, so the back conversion will result in a zero rank stabilizer state.

### Clifford Map Methods

#### Map Embedding

`.embed(small_map, mask)` provides the method to embed a smaller Clifford map on a subset of qubits to the current Clifford map. This is a in-place operation. The Clifford map object that provide this method will get modified under the embedding.

**Parameters:**
* `small_map` is a `CliffordMap` object supported on a subset of qubits.
* `mask` is a boolean array specifying the subset of qubits.

In [13]:
cmap = identity_map(6)
cmap

CliffordMap(
  X0-> +XIIIII
  Z0-> +ZIIIII
  X1-> +IXIIII
  Z1-> +IZIIII
  X2-> +IIXIII
  Z2-> +IIZIII
  X3-> +IIIXII
  Z3-> +IIIZII
  X4-> +IIIIXI
  Z4-> +IIIIZI
  X5-> +IIIIIX
  Z5-> +IIIIIZ)

In [14]:
cmap.embed(random_clifford_map(3), numpy.array([True,False,False,True,True,False]))

CliffordMap(
  X0-> -XIIXII
  Z0-> -IIIZII
  X1-> +IXIIII
  Z1-> +IZIIII
  X2-> +IIXIII
  Z2-> +IIZIII
  X3-> +ZIIZYI
  Z3-> -YIIZII
  X4-> +IIIIYI
  Z4-> -YIIZZI
  X5-> +IIIIIX
  Z5-> +IIIIIZ)

#### Map Composition

`.compose(other)` returns the composition of the current Clifford map with another Clifford map. This will return a new Clifford map without modifying either of the input maps. The Clifford map object which initiates this method will be the preceeding map in the composition. 

**Parameters:**
* `other` - another `CliffordMap`.

Example: composition of a Clifford rotation with its inverse rotation will be an identity map.

In [18]:
clifford_rotation_map('-XXY').compose(clifford_rotation_map('+XXY'))

CliffordMap(
  X0-> +XII
  Z0-> +ZII
  X1-> +IXI
  Z1-> +IZI
  X2-> +IIX
  Z2-> +IIZ)

#### Map Inversion

`.inverse()` returns the inverse of the current Clifford map. This will return a new Clifford map withoutt modifying the original map. The inverse map is such that its composition with the original map must be identity

In [26]:
cmap = clifford_rotation_map('Y')
cmap

CliffordMap(
  X0-> -Z
  Z0-> +X)

In [27]:
cmap.inverse()

CliffordMap(
  X0-> +Z
  Z0-> -X)

Test on random maps.

In [22]:
cmap = random_clifford_map(4)
cmap.inverse().compose(cmap)

CliffordMap(
  X0-> +XIII
  Z0-> +ZIII
  X1-> +IXII
  Z1-> +IZII
  X2-> +IIXI
  Z2-> +IIZI
  X3-> +IIIX
  Z3-> +IIIZ)

In [23]:
cmap.compose(cmap.inverse()) 

CliffordMap(
  X0-> +XIII
  Z0-> +ZIII
  X1-> +IXII
  Z1-> +IZII
  X2-> +IIXI
  Z2-> +IIZI
  X3-> +IIIX
  Z3-> +IIIZ)

Both left and right composition are identity.

### Stabilizer State Methods

#### Measurement

`.measure(obs)` measure the stabilizer state on a set of commuting observables.

**Parameters:**
* `obs` - Observables to measure. The following types are supported:
    * `PauliList` - a list of Pauli operators (user must ensure that operators in the list are commuting, otherwise they can not measured simutaneously).
    * `StabilizerState` - stabilizers of a stabilizer state is always commuting, which can be treated as commuting observables for measurement.
    
**Returns:**
* `out` - measuremnt outcome, can only be $0$, $\pm1$ for independent Pauli observables on stabilizer state.
* `log2prob` - the log2 of the probability of realizing this particular outcome.

<span style="color:red">The following documents is out of date</span>.

## StabilizerState

`StabilizerState(N, r=None, S=None, b=None)` represents a stablizer state for an $[N,r]$ stablizer code (i.e. $N$ physical qubits encoding $r$ logical qubits).

**Parameters**
- `N`: number of physical qubits.
- `r`: number of logical qubits.
- `S`: stabilizer tableau.
- `b`: sign indicator.

Example: A 5-qubit state with 2 logical qubits (being the first 2 physical qubits). The state is stablized by 3 stabilizers (acting on the last 3 qubits).

In [5]:
vaeqst.StabilizerState(5, r=2)

StabilizerState(
 +IIZII
 +IIIZI
 +IIIIZ)

### Representations

#### Density Matrix

A $[N,r]$ stabilizer state is describe by the **density matrix** of the following form:
$$\rho = \frac{1}{2^r}\prod_{k=1}^{N-r}\frac{1+(-)^{b_k}S_k}{2}.$$
* Each stabilizer $S_k$ is a (non-trivial) Pauli operator defined on totally $N$ qubits. The stabilizers commute with each other $[S_k,S_{k'}]=0$. They generate an Abelian subgroup $\mathcal{S}=\{\prod_{k=1}^{N-r} S_k^{a_k}|a_k=0,1\}$ of the $N$-qubit Pauli group, called the *stabilizer group*.
* Each sign indicator $b_k=0,1$ is a binary variable specifying the eigen space of the stabilizer.
* There are totally $N-r$ stabilizers for a $[N,r]$ stablizer code (of code rate: $r/N$). The simultaneous eigenspace of all stabilizers constitutes the *code subspace*. 
* The code subspace is $2^r$ dimensional (which is also the rank of the density matrix $\rho$). The stabilizer state $\rho$ is always defined to be the maximally mixed state in the code subspace, such that $\rho$ is also the **projection operator** that projects any state into the code subspace.

#### Binary Representation of Pauli Operators

Any Pauli operator can be specified by two one-hot (binary) vectors $x$ and $z$ ($x_i,z_i=0,1$ for $i=1,\cdots,N$):
$$\sigma_{(x,z)}=\mathrm{i}^{x\cdot z}\prod_{i=1}^{N}X_i^{x_i}\prod_{i=1}^{N}Z_i^{z_i}.$$
* The binary vector $x$ (or $z$) specifies the qubits where the $X$ (or $Z$) operator acts ($Y$ operator acts at where $X$ and $Z$ act simultaneously).
* **Multiplication** of two Pauli operators
$$\sigma_{(x,z)}\sigma_{(x',z')}=\mathrm{i}^{p(x,z;x',z')}\sigma_{(x+x',z+z')\%2},$$
where the power $p$ of $\mathrm{i}$ in the prefactor is given by
$$p(x,z;x',z')=\sum_{i=1}^{N}\left(z_ix'_i-x_iz'_i + 2(z_i+z'_i)\left\lfloor\frac{x_i+x'_i}{2}\right\rfloor+2(x_i+x'_i)\left\lfloor\frac{z_i+z'_i}{2}\right\rfloor\right)\mod 4.$$
* **Commutation relation**: two Pauli operator either commute to anticommute.
$$\sigma_{(x,z)}\sigma_{(x',z')}=(-)^{c(x,z;x',z')}\sigma_{(x',z')}\sigma_{(x,z)},$$
where the *anticommutation indicator* $c$ has a simpler form
$$c(x,z;x',z')=\frac{p(x,z;x',z')-p(x',z';x,z)}{2}=\sum_{i=1}^{N}\left(z_ix'_i-x_iz'_i\right)\mod 2.$$

The binary vectors $x$ and $z$ can be interweaved into a $2N$-component vector $g=(x_0,z_0,x_1,z_1,\cdots)$, which forms the binary representation of a Pauli operator $\sigma_g$.

#### Stabilizer Tableau

Each stabilizer state is internally stored in the form of a **stabilizer tableau** $S$, together with the **sign indicator** $b$. For a $[N,r]$ stabilizer state, its stabilizer tableau is a $2N\times 2N$ matrix of the following structure

<img src="fig_tableau.png" alt="stabilizer tableau." style="width: 320px;"/>

* Each row is a binary representation $(x,z)$ of a Pauli oparator $\sigma_{(x,z)}$.
* Totally $2N$ Pauli operators grouped into $N$ stabilizers and $N$ destabilizers, such that for $i,j=0,\cdots,2N-1$:
$$\sigma_{g_{i}}\sigma_{g_{j}}=(-)^{\delta_{i+N,j}-\delta_{j+N,i}}\sigma_{g_{j}}\sigma_{g_{i}},$$
i.e. the $i$th stabilizer only anticommute with the $i$th destabilizer and they commute with all the other operators in the tableau.
* The rows $r:N$ correspond to the $N-r$ active stabilizers $S_k$, which stabilize the code subspace (impleted as projection operators). The rows $0:r$ corresponds to the $r$ standby (inactive) stabilizers that does not realy stabilizer the code subspace (but they will act as logical operators in the code subspace).
* The rows $N+r:2N$ correspond to the $N-r$ active destabilizers that anticommute with the active stabilizers. The rows $N:N+r$ correspond to the $r$ standby destabilizers taht anticommute with the standby stabilizers.

Although the stabilizer state is only specified by the active stabilizers, the other operators in the stabilizer tableau are still important in order to complete the operator basis. Such that the tableau can specify an unitary operator in the Clifford group that generate the state. The algorithm must mantain the algebraic structure betwen all stabilizers and destabilizers while updating the tableau.

Example: stabilizer tableau is given by `StabilizerState.S`

In [6]:
rho = vaeqst.StabilizerState(5, r=2)
rho.S

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

The sign indicator is a $N$ vector, which only keeps the sign of the stabilizers. Because the sign of destabilizers are not used in any where.

In [7]:
rho.b

array([0, 0, 0, 0, 0])

### Methods

#### Copy

`StabilizerState.copy()` returns a copy of the state, such that the original state will not be touch by modification on the copy state. It is useful to copy the state for measurement (as measurement changes the state).

Example:

In [25]:
rho0 = vaeqst.StabilizerState(5, r=2)
rho1 = rho0.copy()
rho1.r = 3
rho0, rho1

(StabilizerState(
  +IIZII
  +IIIZI
  +IIIIZ), StabilizerState(
  +IIIZI
  +IIIIZ))

#### Measurement

The stabilizer state can serve both as a density matrix and as a measurement operator.
* When it serves as a density matrix, $(-)^{b_k}S_k$ are the stabilizers that stabilize the state.
* When it serves as a measurment operator, $(-)^{h_k}G_k$ are the commuting observables to be measured in parallel.

`StabilizerState.measure(other)` provides the method to measure on a stabilizer state a set of observables specified by another stabilizer state.

**Parameters:**
* `other`: stabilizer state representing the measurement operator.

**Returns:**
* `out`: the measurement outcome (vector containing outcome of each observable).
* `log2prob`: the log2 probability to obtain this outcome.

**Side Effect:**
The state itself will be updated to the post measurement state.

**Algorithm Outline**:

We scan over every observable $G_k$ in the measurement operator. For each observable, we continue to scan over all operators in the stabilizer tableau. If the observable $G_k$ anticommute with
1. at least one active stabilizer (the first of them being $S_p$) $\to$ $G_k$ is an *error* operator that take the state out of the code subspace $\to$ the measurement will collapse the state to one of the two possible measurement outcomes $G_{k}=\pm 1$ with equal probability, and the state will be *updated*.
2. at least one standby stabilizer or destabilizer (the first of them being $S_p$)$\to$ $G_k$ is a *logical* operator that will further stabilize the code subspace $\to$ the measurement will activate a new pair of stabilizer and destabilizer, and the state will be *extended*.
3. otherwise, $\to$ $G_k$ is a *trivial* operator in the code subspace $\to$ the measurement is classical, and the state is untouched.

|    | `update`? | `extend`? |
|----|-----------|-----------|
| 1. | `True`    | `False`   |
| 2. | `True`    | `True`    |
| 3. | `False`   | `False`   |

* *update*: 
    * $G_k$ must replace $S_p$ to be the active stabilizer. But to mantain its algebraic relatiion with the destabilizer, the original $S_p$ can be promoted to become the corresponding destabilizer. Such that $S_\tilde{p}\leftarrow S_p$, $S_p \leftarrow G_k$ ($\tilde{p}$ denotes the dual row of $p$)
    * The sign of $G_k$ is randomly asigned with half-to-half probability.

* *extend*:
    * The number of logical qubit will be reduced by one $r\leftarrow r-1$.
    * To include new stabilizer-destabilizer pair to the system, apart from the steps in the update algorithm, we also need to bring the new stabilizer $S_p$ to row-$r$ and the new destabilizer $S_{\tilde{p}}$ to row-$(N+r)$. 
    
If update (including extension) did not happen after scanning through all the stabilizers and standby destabilizers, then we know that the measurement is classical ($G_k$ is trivial in code subspace, it must belong to the stabilizer group). So we continue to scan over the remaining active destabilizer. For each active destabilizer $S_j$ that anticommute with $G_k$, it indicates that $G_k$ contains the component of the corresponding active stabilizer $S_{\tilde{j}}$, which should then be collected. Finally the measuremnt outcome $x_k=0,1$ is such that the following equation holds
$$(-)^{h_k+x_k}G_k=\prod_{\tilde{j}}(-)^{b_{\tilde{j}}}S_{\tilde{j}}.$$

Example:

In [8]:
rho = vaeqst.GHZState(5)
print('starting from:\n', rho)
obs = vaeqst.RandomPauliState(5)
print('to measure:\n', obs)
out, log2prob = rho.measure(obs)
print('obtain outcome:\n', out, '\nwith probability 2^({})'.format(log2prob))
print('end up with:\n', rho)

starting from:
 StabilizerState(
 +ZZIII
 +IZZII
 +IIZZI
 +IIIZZ
 +XXXXX)
to measure:
 StabilizerState(
 +XIIII
 -IZIII
 +IIXII
 +IIIXI
 -IIIIX)
obtain outcome:
 [1 0 1 0 1] 
with probability 2^(-5)
end up with:
 StabilizerState(
 -XIIII
 -IIXII
 +IIIXI
 +IIIIX
 -IZIII)


#### Expectation

`StabilizerState.expect(other)` provides the method to compute the expectation value of observables on a stabilizer state.
$$ e_k = (-)^{h_k} \mathrm{Tr} \rho G_k$$

**Parameters:**
* `other`: stabilizer state representing the measurement operator.

**Returns:**
* `expect`: a vector of expectation values $e_k$.

This method will not modify the stabilizer state.

Example:

In [2]:
rho = vaeqst.GHZState(5)
print('base state:\n', rho)
obs = vaeqst.RandomCliffordState(5)
print('measurement:\n', obs)
print('expectation values:\n', rho.expect(obs))

base state:
 StabilizerState(
 +ZZIII
 +IZZII
 +IIZZI
 +IIIZZ
 +XXXXX)
measurement:
 StabilizerState(
 -IYYZX
 -IXYXX
 -ZXZXY
 +ZYYZX
 -ZYZXX)
expectation values:
 [0 0 0 0 0]


The expectation values of Pauli operators on stabilizer states are either $0$ or $\pm1$. The expectation value will be $\pm1$ only if the corresponding observable is in the stabilizer group.

#### Fidelity

`StabilizerState.fidelity(other)` provides the method to compute the fidelity between the base state $\rho$ and another state $\rho'$.

**Parameters:**
* `other`: another stabilizer state $\rho'$ to compare.

**Returns:**
* `F`: fidelity $F(\rho,\rho')=\left(\mathrm{Tr}\sqrt{\sqrt{\rho}\rho'\sqrt{\rho}}\right)^2$.

**Algorithm Outline:**

If both $\rho$ and $\rho'$ are stabilizer states,
$$\rho=\frac{1}{2^{r}}\prod_{j=1}^{N-r}\frac{1+(-)^{b_j}S_j}{2},\quad\rho'=\frac{1}{2^{r'}}\prod_{k=1}^{N-r'}\frac{1+(-)^{h_k}G_{k}}{2}.$$
The stabilizers in $\rho'$ can be classified into three cases on the stabilizer code defined by $\rho$. If $G_k$ anticommute with
1. an active stabilizer in $\rho$: $G_k$ is an *error* operator, denoted by $k\in \mathcal{E}$;
2. a standby stabilizer or destabilizer in $\rho$: $G_k$ is an *logical* operator, denoted by $k\in \mathcal{L}$;
3. other wise, $G_k$ is a *trivial* operator (in the code subspace), denoted by $k\in \mathcal{I}$.

The Fidelity is given by [arXiv:quant-ph/0505036](https://arxiv.org/abs/quant-ph/0505036)
$$F(\rho,\rho')=2^{r-r'-2|\mathcal{L}|-|\mathcal{E}|}\prod_{k\in\mathcal{I}}\frac{1+e_k}{2},$$
where $e_k$ is the expectation value of $(-)^{h_k}G_k$ on $\rho$. The product $o=\prod_{k\in\mathcal{I}}\frac{1+e_k}{2}$ defines the overlap indicator $o=0$ if $\rho$ and $\rho'$ are orthogonal, otherwise $o=1$.

Example: verify that fidelity is symmetric.

In [20]:
rho = vaeqst.GHZState(5)
sig = vaeqst.RandomCliffordState(5)
rho.fidelity(sig), sig.fidelity(rho)

(0.000244140625, 0.000244140625)

Fidelity of a state with itself is always 1, even if the state is mixed.

In [24]:
rho = vaeqst.StabilizerState(5, r=2)
rho.fidelity(rho)

1

#### Entanglement Entropy

`StabilizerState.entropy(A)` calculate the entanglement entropy of the stabilizer state in region $A$.

**Parameters:**
* `A`: a one-hot array of size $N$ specifying the entanglement region. $A_i=1$ if $i\in A$ otherwise $A_i=0$.

**Returns:**
* Entanglement entropy $S_\rho(A)$ in unit of bit (in log2 base).

Example:

In [11]:
A = numpy.array([0,1,0,1,0])
vaeqst.RandomCliffordState(5).entropy(A)

2.0

#### Tokenize

`StabilizerState.tokenize()` tokenize the stabilizer basis. This could be useful for machine learning task. The tokens can be encoded by language processing techniques.

**Rules:**
* 0 = I
* 1 = X
* 2 = Y
* 3 = Z
* 4 = +
* 5 = -
* 6 = (+/-)


Example:

In [20]:
rho = vaeqst.RandomCliffordState(5)
print('state:\n',rho)
rho.tokenize()

state:
 StabilizerState(
 +ZIXZI
 -YIIYX
 -YZXXZ
 +XIYIZ
 +ZYIZX)


array([[3, 0, 1, 3, 0, 4],
       [2, 0, 0, 2, 1, 5],
       [2, 3, 1, 1, 3, 5],
       [1, 0, 2, 0, 3, 4],
       [3, 2, 0, 3, 1, 4]])

#### Sample

`Stabilizer.sample(L)` sample $L$ stabilizers from the stabilizer group, return in the token form.

Example:

In [21]:
rho.sample(3)

array([[0, 3, 1, 3, 2, 5],
       [3, 2, 3, 2, 3, 5],
       [2, 3, 1, 1, 3, 5]])

## RandomCliffordState

`RandomClifordState(N, r=None)` is a subclass of `StabilizerState`. It represents a random stabilizer state generated by unitary transformations sampled uniformly in the global Clifford group.

**Parameters**
- `N`: number of physical qubits.
- `r`: number of logical qubits.

Example:

In [38]:
vaeqst.RandomCliffordState(20)

StabilizerState(
 +ZIIIYIYYXZIXYIZZYYIY
 +XXYZYZXZXIIXZXXYXYYX
 +IXIYIIYXYXZXYYZIZZXX
 +IXIYXYXIXIXXYYIZIYII
 -IZXXYYIYXYXYIIXZXYXI
 -ZYIZYIZZXXXZIZYIIYIX
 -ZIYYZIZZIIIYXIXIIYII
 -IZIIZIZXYZXIZZIIZIYY
 +ZYYIZZYXIYZIXYYXZXIY
 -IZIXXYIZXXZXYZZYYZYZ
 +ZZYXZZXZXIYZYZXYZYIX
 +IZYZZYYIXIZYIIYXIZYX
 -IXZZZZZXIYYZYXXZZXXI
 +YYZZXZIYYIZXIYYZXXII
 +IYZZZZXZYIZIIIYIZYII
 +IIZZYYZYYYYIXIIZYIYI
 -YXZZIXZXIIZIZIIIYXIZ
 +IZIXYZZZXXIXIZYIYYXY
 -ZZZIZXIIXYZXXZIZYYIZ
 -IIIYXIZIZZXXIZYIZZII)

### Random Stabilizer Algorithm

The algorithm generates a random stabilizer tableau, corresponding to a uniformly sampled element in the global Clifford group. The problem can be solved iteratively. 
* Let $(\mathcal{S}_{N-1}, \mathcal{D}_{N-1})$ be the sets of stabilizers and destabilizers (paired up) of $(N-1)$ qubits.
* The sets can be expanded to $N$ qubits by
$$\mathcal{S}_{N}:\left\{\begin{array}{ll}
S_0=U (Z\otimes I^{\otimes (N-1)}) U^\dagger & \\
S_{i+1}=U (I\otimes S'_{i}) U^\dagger & \text{for }S_i\in \mathcal{S}_{N-1}
\end{array}\right.$$
$$\mathcal{D}_{N}:\left\{\begin{array}{ll}
D_0=U (\left\{\begin{array}{c}X\\Y\end{array}\right\}\otimes I^{\otimes (N-1)}) U^\dagger & \\
D_{i+1}=U (I\otimes D'_{i}) U^\dagger & \text{for }D_i\in \mathcal{D}_{N-1}
\end{array}\right.$$
where $U$ is a random Clifford rotation on $N$ qubits.
* $U$ can be generated by first sample a random pair of stabilizer $S_0$ and destablizer $D_0$, and then find the Clifford rotion to diagonalize them to the first qubit.

#### Random Stabilizer-Destabilizer Pair

Using binary representation of Pauli operators,
* Generate a random non-trivial stabilizer $S_0$ by sampling a binary array of $2N$ components, excluding the all-0 case. (If all-0 array is sampled, reject it and resample, until the vector is not all zero).
* Generate a random destabilizer $D_0$ by 
    * first sampling a binary array of $2N$ components,
    * if $D_0$ anticommute with $S_0$: we are done,
    * if $D_0$ commute with $S_0$: pick the first nontrivial qubit on which $S_0$ acts, modify the corresponding $D_0$ operator on that qubit to flip the commutation relation. The modification is given by the following table.
    
| S\D  | I 00 | X 10 | Y 11 | Z 01 |
|------|------|------|------|------|
| X 10 | Z 01 | Y 11 | X 10 | I 00 |
| Y 11 | X 10 | I 00 | Z 01 | Y 11 |
| Z 01 | Y 11 | Z 01 | I 00 | X 10 |

The rule is:
$$x_i^{(D)} \to x_i^{(D)} + z_i^{(S)}, \quad z_i^{(D)}\to z_i^{(D)} + x_i^{(S)} + z_i^{(S)}$$

#### Find Clifford Rotation

To diagonalize $S_0\to Z_0$ and $D_0\to X_0\text{ or }Y_0$.
* First diagonalize $S_0$ by
    * If $S_0$ commute with $Z_0$,
        * If $S_0=I_0\otimes A$, take the first non-trivial qubit of $A$, permute it cyclically among $X,Y,Z$ to create $B$, then
        $$X_0\otimes B: S_0\to X_0\otimes AB$$
        * If $S_0=Z_0\otimes A$:
        $$X_0\otimes A:S_0\to Y_0\otimes I$$
        * Now $S_0$ has been transformed to anticommute with $Z_0$.
    * If $S_0$ anticommute with $Z_0$:
       $$S_0Z_0: S_0\to Z_0\otimes I$$
* Now $S_0=Z_0\otimes I$, $D_0$ must have been transformed to the form $D_0=\begin{array}{c}X_0\\ Y_0\end{array}\otimes C$.
* Then diagonalize $D_0$ by:
$$Z_0\otimes C: Z_0\otimes I\to Z_0\otimes I, D_0\to \begin{array}{c}X_0\\ Y_0\end{array}\otimes I.$$

Collect the Clifford roations along the way and apply them in reverse order to scramble the stabilizers in $(\mathcal{S}_{N-1}, \mathcal{D}_{N-1})$ to the $N$ qubit system.

## RandomPauliState

`RandomPauliState(N, r=None)` is a subclass of `StabilizerState`. It represents a random stabilizer state generated by unitary transformations sampled uniformly in the local (onsite) Clifford group.

**Parameters**
- `N`: number of physical qubits.
- `r`: number of logical qubits.

Example:

In [39]:
vaeqst.RandomPauliState(20)

StabilizerState(
 +YIIIIIIIIIIIIIIIIIII
 +IZIIIIIIIIIIIIIIIIII
 -IIZIIIIIIIIIIIIIIIII
 -IIIYIIIIIIIIIIIIIIII
 -IIIIZIIIIIIIIIIIIIII
 -IIIIIYIIIIIIIIIIIIII
 -IIIIIIXIIIIIIIIIIIII
 -IIIIIIIZIIIIIIIIIIII
 +IIIIIIIIXIIIIIIIIIII
 +IIIIIIIIIZIIIIIIIIII
 -IIIIIIIIIIXIIIIIIIII
 +IIIIIIIIIIIXIIIIIIII
 -IIIIIIIIIIIIYIIIIIII
 +IIIIIIIIIIIIIZIIIIII
 -IIIIIIIIIIIIIIYIIIII
 +IIIIIIIIIIIIIIIZIIII
 -IIIIIIIIIIIIIIIIYIII
 +IIIIIIIIIIIIIIIIIYII
 -IIIIIIIIIIIIIIIIIIXI
 +IIIIIIIIIIIIIIIIIIIY)

## GHZState

`GHZState(N)` is a subclass of `StabilizerState`. It represents a GHZ state $\rho=|\Psi\rangle\langle\Psi|$, with $|\Psi\rangle=\frac{1}{\sqrt{2}}(|00\cdots 0\rangle+|11\cdots 1\rangle)$.

**Parameters**
- `N`: number of physical qubits.

Example:

In [40]:
vaeqst.GHZState(20)

StabilizerState(
 +ZZIIIIIIIIIIIIIIIIII
 +IZZIIIIIIIIIIIIIIIII
 +IIZZIIIIIIIIIIIIIIII
 +IIIZZIIIIIIIIIIIIIII
 +IIIIZZIIIIIIIIIIIIII
 +IIIIIZZIIIIIIIIIIIII
 +IIIIIIZZIIIIIIIIIIII
 +IIIIIIIZZIIIIIIIIIII
 +IIIIIIIIZZIIIIIIIIII
 +IIIIIIIIIZZIIIIIIIII
 +IIIIIIIIIIZZIIIIIIII
 +IIIIIIIIIIIZZIIIIIII
 +IIIIIIIIIIIIZZIIIIII
 +IIIIIIIIIIIIIZZIIIII
 +IIIIIIIIIIIIIIZZIIII
 +IIIIIIIIIIIIIIIZZIII
 +IIIIIIIIIIIIIIIIZZII
 +IIIIIIIIIIIIIIIIIZZI
 +IIIIIIIIIIIIIIIIIIZZ
 +XXXXXXXXXXXXXXXXXXXX)